## 3ra pregunta

Del dataset https://archive.ics.uci.edu/ml/datasets/Credit+Approval utilice al menos un 80% de registros categorizados por una clase para entrenar. Obtenga una red neuronal con la librería scikit-learn. Obtenga la matriz de confusión.

In [52]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, make_scorer
from sklearn.model_selection import cross_val_score, GridSearchCV, StratifiedKFold, train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import numpy as np
import pandas as pd

### Lectura del Dataset

Mediante pandas realziamos la carga del dataset, aprovación de creditos

In [62]:
data = pd.read_csv("./crx.data", header=None)
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


### Preprocesamiento

De los datos cuantitativos, mediante la función `describe()` propia de `pandas` vemos los estadisticos mas significativos para nuestro dataset

In [63]:
data.describe()

,2,7,10,14
count,690.000000,690.000000,690.00000,690.000000
mean,4.758725,2.223406,2.40000,1017.385507
std,4.978163,3.346513,4.86294,5210.102598
min,0.000000,0.000000,0.00000,0.000000
25%,1.000000,0.165000,0.00000,0.000000
50%,2.750000,1.000000,0.00000,5.000000
75%,7.207500,2.625000,3.00000,395.500000
max,28.000000,28.500000,67.00000,100000.000000


Antes realizamos la limpieza del dataset, eliminando valores arbitrarios, que en la observación del dataset podemos apreciar. reemplazando estos por valores de:

* La media, si se trata de valores cuantitativos
* La moda, si se trata de valores cualitativos

In [64]:
data = data.replace("?", np.NaN)
data = data.fillna(data.mean())

for column in data.columns:
    if data[column].dtypes == 'object':
        data[column] = data[column].fillna(data[column].value_counts().index[0])

Primero, convertiremos todos los valores no numéricos en valores numéricos. Hacemos esto porque no solo da como resultado un cálculo más rápido, sino que también muchos modelos de aprendizaje automático (y especialmente los desarrollados con scikit-learn) requieren que los datos estén en un formato estrictamente numérico.

In [65]:
le = LabelEncoder()

for column in data.columns:
    if data[column].dtype=='object':
        data[column]=le.fit_transform(data[column])

data

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1,156,0.000,1,0,12,7,1.250,1,1,1,0,0,68,0,0
1,0,328,4.460,1,0,10,3,3.040,1,1,6,0,0,11,560,0
2,0,89,0.500,1,0,10,3,1.500,1,0,0,0,0,96,824,0
3,1,125,1.540,1,0,12,7,3.750,1,1,5,1,0,31,3,0
4,1,43,5.625,1,0,12,7,1.710,1,0,0,0,2,37,0,0
5,1,168,4.000,1,0,9,7,2.500,1,0,0,1,0,115,0,0
6,1,179,1.040,1,0,11,3,6.500,1,0,0,1,0,54,31285,0
7,0,74,11.585,1,0,2,7,0.040,1,0,0,0,0,23,1349,0
8,1,310,0.500,2,2,8,3,3.960,1,0,0,0,0,62,314,0
9,1,255,4.915,2,2,12,7,3.165,1,0,0,1,0,15,1442,0


Ademásvemos que columnas como `DriversLicense` y `ZipCode` no son tan importantes como las otras columnas del conjunto de datos para predecir las aprobaciones de tarjetas de crédito. Deberíamos descartarlos para diseñar nuestro modelo de aprendizaje automático con el mejor conjunto de características. Esto a menudo se denomina ingeniería de características o, más específicamente, selección de características.

In [66]:
data = data.drop([data.columns[10],data.columns[13]], axis=1)
data.head()

,0,1,2,3,4,5,6,7,8,9,11,12,14,15
0,1,156,0.000,1,0,12,7,1.25,1,1,0,0,0,0
1,0,328,4.460,1,0,10,3,3.04,1,1,0,0,560,0
2,0,89,0.500,1,0,10,3,1.50,1,0,0,0,824,0
3,1,125,1.540,1,0,12,7,3.75,1,1,1,0,3,0
4,1,43,5.625,1,0,12,7,1.71,1,0,0,2,0,0


In [67]:
data = data.values
X,y = data[:,0:13], data[:,13]

scaler = MinMaxScaler(feature_range=(0,1))
rescaledX = scaler.fit_transform(X)

Primero, dividiremos nuestros datos en conjuntos de entrenamiento y conjuntos de pruebas para preparar nuestros datos para dos fases diferentes del modelado de aprendizaje automático: entrenamiento y pruebas.

In [68]:
X_train, X_test, y_train, y_test = train_test_split(rescaledX, y, test_size=0.20, random_state=42)

Entrenando el modelo, con una red neuronal, de tipo multilayer Perceptron

In [89]:
clf = MLPClassifier(hidden_layer_sizes=(200,150,50), solver='adam', random_state=1, max_iter=500, activation='relu')
clf.fit(X_train, y_train)
Y_pred = clf.predict(X_test)

Ahora evaluaremos nuestro modelo en el conjunto de prueba con respecto a la precisión de clasificación. Pero también echaremos un vistazo a la matriz de confusión del modelo. En el caso de predecir aplicaciones de tarjetas de crédito, es igualmente importante ver si nuestro modelo de aprendizaje automático es capaz de predecir el estado de aprobación de las aplicaciones como denegadas que originalmente fueron denegadas. Si nuestro modelo no está funcionando bien en este aspecto, entonces podría terminar aprobando la aplicación que debería haber sido aprobada. La matriz de confusión nos ayuda a ver el desempeño de nuestro modelo desde estos aspectos.

In [90]:
print(confusion_matrix(y_test, Y_pred))
print("Exactitud: ", accuracy_score(y_test, Y_pred) * 100, "%")

[[57 13]
 [10 58]]
Exactitud:  83.33333333333334 %
